# Extract data

# Setup

In [1]:
# %pip install -r requirements.txt

In [2]:
# Environment Variables
import os
from dotenv import load_dotenv

# Load env
load_dotenv()

True

# Digital Ocean

In [3]:
# Set env path
os.environ["DO_USERNAME"] = "doadmin"
os.environ["DO_PASSWORD"] = "law-gpt-postgresql-cluster-do-user-15523133-0.c.db.ondigitalocean.com"
os.environ["DO_HOST"] = "host"
os.environ["DO_PORT"] = "25060"
os.environ["DO_DATABASE"] = "document_keys"
os.environ["DO_SSL_MODE"] = "require"

# Google

In [4]:
# Set env path
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "lawgpt-410122-140fba6fba7b.json"

# Bucket params
bucket_name = 'lawgpt_madrid_bucket_1'

# Directory

In [5]:
# Set directory to file location
from pathlib import Path
import sys
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)
# Get the current working directory
current_directory = os.getcwd()
current_directory

'/notebooks/LawGPT'

# Libraries

In [6]:
# General libraries
import pandas as pd
import numpy as np
import datetime
import time
import csv
import os

# Scrapping
from bs4 import BeautifulSoup
import unicodedata
import unidecode
import requests
import string
import json
import lxml
import xml
import bs4
import gc
import re

# Langchain
from langchain.text_splitter import CharacterTextSplitter

# Multiprocessing
from multiprocessing import Pool, Manager, Queue, Process
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Value, Lock
from functools import partial

# Cloud
from google.cloud import storage

# Other
from tqdm.notebook import tqdm
import logging

# Local
from functions import *

# Warnings
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Start timing the notebook run
start_time = time.time()

# Keywords

In [8]:
key_words = ['constitucion', 'legislacion', 'ley', 'codigo', 'estatuto', 'derecho', 'decreto', 'norma', 
             'deber', 'defensa', 'seguridad', 'proteccion', 'defensor', 'jurisprudencia', 'estado', 
             'autonomia', 'congreso', 'senado', 'parlamento', 'nacional', 'local', 'suspension', 'articulo', 
             'europa', 'reguladora', 'impuesto', 'penal', 'civil', 'laboral', 'convenios', 'pleno']

In [9]:
key_words = ['codigo penal']

# BOE: Boletín Oficial del Estado

In [10]:
# Parameters
start_date = datetime.date(1978, 1, 1)

# Define the end date to extract
end_date = datetime.datetime.now().date()

# BOE Class
boe_class = ['A', 'C']

# Years to extract
years = [str(i) for i in range(1978, 2025)]

# Path to store full extraction
folder_path = 'raw_data/boe_year/'

# Multiprocessing
# n_cores = os.cpu_count() - 2
n_cores = 1
pool = Pool(processes = n_cores)

# 1. Find local files

In [11]:
# Params
file_list = ["boe_ids.csv"]
local_folder_path = "raw_data/"

# List CSV files locally
local_csv_files = list_csv_files(local_folder_path)
local_csv_files

['boe_ids.csv']

# 2. Identify existing URLs from daily summaries

In [12]:
# BOE IDs filename
boe_ids_filename = 'raw_data/boe_ids.csv'

In [ ]:
# Update BOE IDs
boe_ids = extract_boe_ids(start_date, end_date, boe_class, boe_ids_filename)

Processing days:   0%|          | 0/14437 [00:00<?, ?it/s]

In [ ]:
# Show
boe_ids.head()

In [ ]:
# Length of file
len(boe_ids)

In [ ]:
# Upload the file to Google Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(boe_ids_filename)
blob.upload_from_filename(boe_ids_filename)

# 3. Identify relevant keywords from extracted URLs and titles

In [ ]:
# BOE filtered IDs filename
boe_filtered_ids_name = 'raw_data/boe_filtered_ids.csv'

In [ ]:
# Identify with keywords
boe_filtered_ids = find_titles_with_keywords(boe_ids, key_words)

In [ ]:
# Show
boe_filtered_ids.head()

In [ ]:
# Length of file
len(boe_filtered_ids)

In [ ]:
# Save output
boe_filtered_ids.to_csv(boe_filtered_ids_name, index = False)

In [ ]:
# Upload the file to Google Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(boe_filtered_ids_name)
blob.upload_from_filename(boe_filtered_ids_name)

# 4. Extract publications from identified URLs

In [ ]:
# Initialize multiprocessing manager and queue for logging
manager = Manager()
log_queue = manager.Queue()

# Start logger process
logger_process = Process(target=setup_logger, args=(log_queue,))
logger_process.start()

# Start extraction process
pool = Pool(processes=n_cores)
processes = [pool.apply_async(extract_boe_year, args=(folder_path, boe_filtered_ids, year, log_queue)) for year in years]
[p.get() for p in processes]

# Signal the logger process to stop
log_queue.put(None)
logger_process.join()

# 5. Append results

In [ ]:
# List CSV files
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty DataFrame to store the concatenated data
boe_concat = pd.DataFrame()

# Loop through the CSV files and concatenate them
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    boe_concat = pd.concat([boe_concat, df], ignore_index=True)

In [ ]:
# Show
boe_concat.head()

In [ ]:
# Length of file
len(boe_concat)

# 6. Format results

In [ ]:
# BOE format filename
boe_format_name = 'raw_data/boe_data.csv'

In [ ]:
# Input for formatting
boe_format = boe_concat.copy()

# Create id column
boe_format['date'] =  pd.to_datetime(boe_format['id'], format='%Y%m%d')

# Drop column
boe_format = boe_format.drop(['id'], axis = 1)

# Regular expression to capture the year and the ID
regex_pattern = r'BOE-[ABC]-(\d{4})-(\d+)'

# Extract the year and ID and concatenate them into a new column
boe_format['id'] = boe_format['url'].str.extract(regex_pattern).agg('-'.join, axis=1)

# Filter columns
cols = ['id', 'url', 'title', 'date', 'legislative_origin', 'department', 'rang', 'text']
boe_format = boe_format[cols]

# Sort by date
boe_format = boe_format.sort_values(by = 'date', ascending = True)

In [ ]:
# Show
boe_format.head()

In [ ]:
# Length of file
len(boe_format)

In [ ]:
# Save output
boe_format.to_csv(boe_format_name, index=False)

In [ ]:
# Upload the file to Google Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(boe_format_name)
blob.upload_from_filename(boe_format_name)

# Runtime

In [ ]:
# End time of notebook run
end_time = time.time()
elapsed_time = end_time - start_time

# Convert elapsed time to hours and minutes
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)

# Print the result
print(f"Time elapsed: {hours} hours and {minutes} minutes.")

# Clean

In [ ]:
# Clean
gc.collect()